In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import math
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.optim as optim
print(f"PyTorch version: {torch.__version__}")
print(f"Device available: {'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")

PyTorch version: 2.7.1+cu128
Device available: GPU (CUDA)


### Problem
- 1D line of length $L'$ with per-unit-length $(R',L',G',C')$.
- Source placed at $z=0$ with $(V_{s},Z_{s})$
- Unknown load $Z_{L}$ at $z=L$

### Goal
- Infer $V(z,t), I((z,t)$ everywhere
- Estimate $(R',L',G',C')$ from sparse time traces 

In [3]:
# ready-to-run script outline
import torch, math
L, T = 1.0, 20e-9
device='cpu'
class Net(torch.nn.Module):
    def __init__(self): 
        super().__init__(); self.layers=torch.nn.Sequential(
            torch.nn.Linear(2,128), torch.nn.Tanh(),
            torch.nn.Linear(128,128), torch.nn.Tanh(),
            torch.nn.Linear(128,2))
        self.raw = torch.nn.Parameter(torch.tensor([ -3.0, -1.0, -8.0, -12.0, 3.0])) # log-params
    def forward(self, zt): return self.layers(zt)
    def params_phys(self):
        R = torch.nn.functional.softplus(self.raw[0]); Lp = torch.nn.functional.softplus(self.raw[1])
        G = torch.nn.functional.softplus(self.raw[2]); Cp = torch.nn.functional.softplus(self.raw[3])
        ZL = torch.nn.functional.softplus(self.raw[4])+1e-3
        return R, Lp, G, Cp, ZL

net=Net().to(device)
opt=torch.optim.Adam(net.parameters(), lr=1e-3)

# collocation
Nz,Nt=256,256
z=torch.rand(Nz*Nt,1)*L; t=torch.rand(Nz*Nt,1)*T
zt=torch.cat([z,t],1).to(device); zt.requires_grad_(True)

# source
Zs=50.0
def Vs(t): return torch.sin(2*math.pi*1e9*t)  # 1 GHz tone

for it in range(5000):
    Vhat,Ihat = net(zt).split(1,dim=1)
    dV_dz = torch.autograd.grad(Vhat, zt, torch.ones_like(Vhat), retain_graph=True, create_graph=True)[0][:,0:1]
    dI_dz = torch.autograd.grad(Ihat, zt, torch.ones_like(Ihat), retain_graph=True, create_graph=True)[0][:,0:1]
    dV_dt = torch.autograd.grad(Vhat, zt, torch.ones_like(Vhat), retain_graph=True, create_graph=True)[0][:,1:2]
    dI_dt = torch.autograd.grad(Ihat, zt, torch.ones_like(Ihat), retain_graph=True, create_graph=True)[0][:,1:2]

    R,Lp,G,Cp,ZL = net.params_phys()
    pde1 = dV_dz + Lp*dI_dt + R*Ihat
    pde2 = dI_dz + Cp*dV_dt + G*Vhat

    # boundary samples
    tb = torch.rand(1024,1)*T
    z0 = torch.zeros_like(tb); zL = torch.ones_like(tb)*L
    V0,I0 = net(torch.cat([z0,tb],1))
    VL,IL = net(torch.cat([zL,tb],1))
    bc0 = V0 - (Vs(tb) - Zs*I0)
    bcL = VL - ZL*IL

    loss = (pde1.pow(2).mean()+pde2.pow(2).mean()
           +1e2*bc0.pow(2).mean()+1e2*bcL.pow(2).mean())

    opt.zero_grad(); loss.backward(); opt.step()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\jones\anaconda3\envs\CompEnv\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jones\AppData\Local\Temp\ipykernel_29508\2632919541.py", line 45, in <module>
    V0,I0 = net(torch.cat([z0,tb],1))
    ^^^^^
ValueError: too many values to unpack (expected 2)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\jones\anaconda3\envs\CompEnv\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jones\anaconda3\envs\CompEnv\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jones\anaconda3\envs\CompEnv\